In [0]:
!git clone https://github.com/pdhruv93/YOLO-Face-Recognition-DLIB.git

Cloning into 'YOLO-Object-Detection'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 23 (delta 4), reused 22 (delta 3), pack-reused 0
Unpacking objects: 100% (23/23), done.


In [0]:
%cd YOLO-Object-Detection//yolo-object-detection

/content/YOLO-Object-Detection/yolo-object-detection/videos/YOLO-Object-Detection/yolo-object-detection


In [0]:
!wget "https://pjreddie.com/media/files/yolov3.weights"

--2019-02-25 21:46:33--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.3.39
Connecting to pjreddie.com (pjreddie.com)|128.208.3.39|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  42.9MB/s    in 5.4s    

2019-02-25 21:46:39 (43.9 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [0]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [0]:
videoInPath="videos//car_chase_01.mp4"
videoOutPath="videos//car_chase_01_yolo.avi"
yoloPath="yolo-coco"
# command line arguments in dict form
#video-- path to input video
#yolo --base path to YOLO directory
#confidence --minimum probability to filter weak detections
#threshold --threshold when applyong non-maxima suppression
args={'videoIn': videoInPath , 'videoOut': videoOutPath , 'yolo': yoloPath , 'confidence': 0.5, 'threshold': 0.3}

In [0]:
# load the COCO class labels our YOLO model was trained on
LABELS = open(yoloPath+"//coco.names").read().strip().split("\n")
print("Toal classes {0}".format(len(LABELS)))

Toal classes 80


In [0]:
# initialize a list of colors to represent each possible class label
np.random.seed(42)
#create random list of int type numbers from range 0-255. Size = len(LABELS), 3... 3 is for RGB
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),dtype="uint8")

In [0]:
# derive the paths to the YOLO weights and model configuration
weightsPath = "yolov3.weights"
configPath = yoloPath+"//yolov3.cfg"
# load our YOLO object detector trained on COCO dataset (80 classes)
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

In [0]:
#Generally in a sequential CNN network there will be only one output layer at the end. 
#In the YOLO v3 architecture we are using there are multiple output layers giving out predictions.
ln_all = net.getLayerNames()
#print(ln)

ln=[]
# determine only the *output* layer names that we need from YOLO
for i in ln_all:
    if "yolo" in i:
        ln.append(i)
        
print(ln)

['yolo_82', 'yolo_94', 'yolo_106']


In [0]:
#Open a file pointer to the video file
vs = cv2.VideoCapture(args["videoIn"])

writer = None #Initialize video writer
(W, H) = (None, None) #Initialize frame dimensions

# try to determine the total number of frames in the video file
try:
    total = int(vs.get(cv2.CAP_PROP_FRAME_COUNT))
    print("[INFO] {} total frames in video".format(total))
except:
    # an error occurred while trying to determine the total number of frames in the video file
    print("[INFO] could not determine # of frames in video")
    print("[INFO] no approx. completion time can be provided")
    total = -1

currentFrameNumber=1

try:
    # loop over frames from the video file stream
    while True:
        # read the next frame from the file
        (grabbed, frame) = vs.read()

        # if the frame was not grabbed, then we have reached the end of the stream
        if not grabbed:
            break

        #break if current frame count exceeds total count
        if(currentFrameNumber > total):
            break

        #break if q from keyboard is read
        if(cv2.waitKey(1) == ord('q')):
            break

        print("Grabbed frame.....{}".format(currentFrameNumber))

        # if the frame dimensions are empty, grab them
        if W is None or H is None:
            (H, W) = frame.shape[:2]

        # construct a blob from the input frame
        #why we do : preprocessing images and preparing them for classification via pre-trained deep learning models.
        blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),swapRB=True, crop=False)

        net.setInput(blob)

        #perform a forward pass of the YOLO object detector, giving us our bounding boxes and associated probabilities
        layerOutputs = net.forward(ln)

        #yolo has now processed our image and got the data. we now just need to fetch that data
        # initialize empty lists
        boxes = []
        confidences = []
        classIDs = []

        #Let’s begin populating these lists with data from our YOLO layerOutputs
        # loop over each of the layer outputs
        for output in layerOutputs:
            # loop over each of the detections
            for detection in output:
                # extract the class ID and confidence (i.e., probability) of the current object detection
                scores = detection[5:]
                classID = np.argmax(scores)
                confidence = scores[classID]

                # filter out weak predictions by ensuring the detected probability is greater than the minimum probability
                if confidence > args["confidence"]:
                    # scale the bounding box coordinates back relative to the size of the image
                    #YOLO returns the center (x, y)-coordinates of the bounding box followed by the box width and height
                    box = detection[0:4] * np.array([W, H, W, H])

                    #astype("int") witll convert box values to int values
                    (centerX, centerY, width, height) = box.astype("int")

                    # use the center (x, y)-coordinates to derive the top-left corner of the bounding box
                    x = int(centerX - (width / 2))
                    y = int(centerY - (height / 2))

                    # update our list of bounding box coordinates, confidences and class IDs
                    boxes.append([x, y, int(width), int(height)])
                    confidences.append(float(confidence))
                    classIDs.append(classID)

        # apply non-maxima suppression to suppress weak, overlapping bounding boxes
        #All that is required is that we submit our:
        #bounding boxes , confidences , confidence threshold and NMS threshold
        idxs = cv2.dnn.NMSBoxes(boxes, confidences, args["confidence"],args["threshold"])

        #idxs now hold indexes after non maxima suppression


        #draw the boxes and class text on the frame
        if len(idxs) > 0:
            for i in idxs.flatten():
                # extract the bounding box coordinates
                (x, y) = (boxes[i][0], boxes[i][1]) #x,y--coordinate of top left corner
                (w, h) = (boxes[i][2], boxes[i][3])

                #pick the color
                color = [int(c) for c in COLORS[classIDs[i]]]

                # draw a bounding box rectangle
                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)  #2 is the line thickness

                #prepare text
                text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])

                #put text on image at x, y-5....a bit up then the box
                cv2.putText(frame, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,0.5, color, 2)

        # check if the video writer is None
        if writer is None:
            # initialize our video writer
            fourcc = cv2.VideoWriter_fourcc(*"MJPG")

            writer = cv2.VideoWriter(args["videoOut"], fourcc, 30,(frame.shape[1], frame.shape[0]), True)

        # write the output frame to disk
        writer.write(frame)

        currentFrameNumber=currentFrameNumber+1

except: 
  pass
    
# release the file pointers
print("[INFO] cleaning up...")

if writer is not None:
  writer.release()
  vs.release()

[INFO] 350 total frames in video
Grabbed frame.....1
Grabbed frame.....2
Grabbed frame.....3
Grabbed frame.....4
Grabbed frame.....5
Grabbed frame.....6
Grabbed frame.....7
Grabbed frame.....8
Grabbed frame.....9
Grabbed frame.....10
Grabbed frame.....11
Grabbed frame.....12
Grabbed frame.....13
Grabbed frame.....14
Grabbed frame.....15
Grabbed frame.....16
Grabbed frame.....17
Grabbed frame.....18
Grabbed frame.....19
Grabbed frame.....20
Grabbed frame.....21
Grabbed frame.....22
Grabbed frame.....23
Grabbed frame.....24
Grabbed frame.....25
Grabbed frame.....26
Grabbed frame.....27
Grabbed frame.....28
Grabbed frame.....29
Grabbed frame.....30
Grabbed frame.....31
Grabbed frame.....32
Grabbed frame.....33
Grabbed frame.....34
Grabbed frame.....35
Grabbed frame.....36
Grabbed frame.....37
Grabbed frame.....38
Grabbed frame.....39
Grabbed frame.....40
Grabbed frame.....41
Grabbed frame.....42
Grabbed frame.....43
Grabbed frame.....44
Grabbed frame.....45
Grabbed frame.....46
Grabbed fr

In [0]:
#code for saving the output video
%cd videos
from google.colab import files
files.download('car_chase_01_yolo.avi')

/content/YOLO-Object-Detection/yolo-object-detection/videos/YOLO-Object-Detection/yolo-object-detection/videos
